In [7]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor, Compose, Resize, Normalize
from sklearn.model_selection import train_test_split, StratifiedKFold, StratifiedGroupKFold
from torch.utils.data import Subset
from sklearn.metrics import f1_score

In [8]:
# Configuration
DATA_DIR = "data"
BATCH_SIZE = 32
EPOCHS = 10
IMG_SIZE = (128, 128)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
N_SPLITS = 5  # Number of folds for cross-validation

In [9]:
# Transformations
transform = Compose([
    Resize(IMG_SIZE),
    ToTensor(),
    Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [10]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(SimpleCNN, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        
        # Pooling layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Calculate the size after convolutions and pooling
        # Input size: IMG_SIZE[0] x IMG_SIZE[1]
        # After first conv+pool: IMG_SIZE[0]/2 x IMG_SIZE[1]/2
        # After second conv+pool: IMG_SIZE[0]/4 x IMG_SIZE[1]/4
        conv_output_size = 32 * (IMG_SIZE[0] // 4) * (IMG_SIZE[1] // 4)
        
        # Fully connected layers
        self.fc1 = nn.Linear(conv_output_size, 128)
        self.fc2 = nn.Linear(128, num_classes)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # -> [B, 16, IMG_SIZE[0]/2, IMG_SIZE[1]/2]
        x = self.pool(F.relu(self.conv2(x)))  # -> [B, 32, IMG_SIZE[0]/4, IMG_SIZE[1]/4]
        x = x.view(x.size(0), -1)             # flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [11]:
# Simple CNN Model
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, 
                      kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * (IMG_SIZE[0] // 4) * (IMG_SIZE[1] // 4), 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

In [ ]:
# Load full dataset
dataset = ImageFolder(root=DATA_DIR, transform=transform)

# Get indices and labels for stratified split
indices = list(range(len(dataset)))
labels = [dataset.targets[i] for i in indices]
# train_indices, test_indices = train_test_split(indices, train_size=0.8, stratify=labels, random_state=42)


# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

# Lists to store metrics for each fold
fold_metrics = []

# Training and evaluation for each fold
for fold, (train_idx, val_idx) in enumerate(skf.split(indices, labels)):
    print(f"\nFold {fold + 1}/{N_SPLITS}")
    
    # Create data loaders for this fold
    train_subsampler = Subset(dataset, train_idx)
    val_subsampler = Subset(dataset, val_idx)
    
    train_loader = DataLoader(train_subsampler, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_subsampler, batch_size=BATCH_SIZE, shuffle=False)
    
    # Initialize model for this fold
    num_classes = len(dataset.classes)
    model = SimpleCNN(num_classes=num_classes).to(DEVICE)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Training loop (your existing training code)
    for epoch in range(EPOCHS):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        print(f"Epoch [{epoch+1}/{EPOCHS}], Loss: {running_loss/len(train_loader):.4f}")
    
    # Validation loop
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    # Calculate metrics for this fold
    avg_val_loss = val_loss / len(val_loader)
    accuracy = 100 * correct / total
    f1 = f1_score(all_labels, all_preds, average='weighted')
    
    # Store metrics for this fold
    fold_metrics.append({
        'fold': fold + 1,
        'running_loss': running_loss/len(train_loader),
        'val_loss': avg_val_loss,
        'val_accuracy': accuracy,
        'f1_score': f1
    })
    
    print(f"Fold {fold + 1} Results:")
    print(f"Validation Loss: {avg_val_loss:.4f}")
    print(f"Validation Accuracy: {accuracy:.2f}%")
    print(f"F1 Score: {f1:.4f}")

# Convert metrics to DataFrame
metrics_df = pd.DataFrame(fold_metrics)

# Add mean and std of metrics
mean_metrics = metrics_df.mean()
std_metrics = metrics_df.std()

print("\nCross-validation Results:")
print(f"Average Loss: {mean_metrics['val_loss']:.4f} ± {std_metrics['val_loss']:.4f}")
print(f"Average Accuracy: {mean_metrics['val_accuracy']:.2f}% ± {std_metrics['val_accuracy']:.2f}%")
print(f"Average F1 Score: {mean_metrics['f1_score']:.4f} ± {std_metrics['f1_score']:.4f}")

# Save metrics to CSV
try:
    existing_df = pd.read_csv('model_metrics.csv')
    metrics_df = pd.concat([existing_df, metrics_df], ignore_index=True)
except FileNotFoundError:
    pass

metrics_df.to_csv('model_metrics.csv', index=False)


Fold 1/5


/home/jjalonzo-estra/miniconda3/envs/Jotun/lib/python3.12/site-packages/PIL/Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch [1/10], Loss: 3.4581
Epoch [2/10], Loss: 2.1563
Epoch [3/10], Loss: 1.5103
Epoch [4/10], Loss: 1.0013
Epoch [5/10], Loss: 0.6176
Epoch [6/10], Loss: 0.3970
Epoch [7/10], Loss: 0.2776
Epoch [8/10], Loss: 0.2277
Epoch [9/10], Loss: 0.1864
Epoch [10/10], Loss: 0.1650
Fold 1 Results:
Validation Loss: 4.0888
Validation Accuracy: 51.19%
F1 Score: 0.5132

Fold 2/5


/home/jjalonzo-estra/miniconda3/envs/Jotun/lib/python3.12/site-packages/PIL/Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [32]:
# Artifact
# # Load full dataset
# dataset = ImageFolder(root=DATA_DIR, transform=transform)

# # Get indices and labels for stratified split
# indices = list(range(len(dataset)))
# labels = [dataset.targets[i] for i in indices]
# # Create subset of dataset
# train_dataset = torch.utils.data.Subset(dataset, train_indices)
# train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
# test_dataset = torch.utils.data.Subset(dataset, test_indices)
# test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [34]:
# # Initialize model
# num_classes = len(dataset.classes)
# model = CNN(num_classes=num_classes).to(DEVICE)

# # Loss and optimizer
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# # Training loop
# for epoch in range(1):
#     model.train()
#     running_loss = 0.0
#     for images, labels in train_loader:
#         images, labels = images.to(DEVICE), labels.to(DEVICE)

#         outputs = model(images)
#         loss = criterion(outputs, labels)

#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()

#     print(f"Epoch [{epoch+1}/{EPOCHS}], Loss: {running_loss/len(train_loader):.4f}")

In [ ]:
# # Testing loop
# model.eval()  # Set model to evaluation mode
# test_loss = 0.0
# correct = 0
# total = 0

# with torch.no_grad():  # Disable gradient computation for testing
#     for images, labels in test_loader:
#         images, labels = images.to(DEVICE), labels.to(DEVICE)
        
#         outputs = model(images)
#         loss = criterion(outputs, labels)
        
#         test_loss += loss.item()
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

# # Calculate and print test metrics
# avg_test_loss = test_loss / len(test_loader)
# accuracy = 100 * correct / total

# # Calculate F1 score

# f1 = f1_score(labels.cpu().numpy(), predicted.cpu().numpy(), average='weighted')

# print(f"Test Loss: {avg_test_loss:.4f}")
# print(f"Test Accuracy: {accuracy:.2f}%")
# print(f"F1 Score: {f1:.4f}")


In [38]:
# # Create a dictionary to store metrics
# metrics = {
#     'running_loss': running_loss/len(train_loader),
#     'test_loss': avg_test_loss,
#     'test_accuracy': accuracy,
#     'f1_score': f1
# }

# # Convert to DataFrame and save to CSV
# import pandas as pd
# metrics_df = pd.DataFrame([metrics])

# # Check if file exists and append or create new
# try:
#     existing_df = pd.read_csv('model_metrics.csv')
#     metrics_df = pd.concat([existing_df, metrics_df], ignore_index=True)
# except FileNotFoundError:
#     pass

# metrics_df.to_csv('model_metrics.csv', index=False)
